# Milestone 2

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import os, zipfile , json , random, requests
import re
from pathlib import Path
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import layers
from tensorflow.keras import  Sequential
import string

2025-04-22 20:32:56.291554: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745353976.473503      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745353976.524668      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Explorting dataset:

In [2]:

def is_kaggle():
    # Kaggle kernels always set this env var
    return 'KAGGLE_URL_BASE' in os.environ

def is_colab():
    return (not is_kaggle()) and os.path.exists('/content')

def maybe_mount_drive():
    if is_colab():
        from google.colab import drive
        if not os.path.isdir('/content/drive'):
            drive.mount('/content/drive')

def get_data_path():
    if is_kaggle():
        return '/kaggle/input/squad-2-0/'
    elif is_colab():
        return '/content/drive/MyDrive/SQuAD'
    else:
        return './data/'
def get_model_dir():
    if is_colab():
        model_dir = '/content/drive/MyDrive/models'
    elif is_kaggle():
        model_dir = '/kaggle/working/models'
    else:
        model_dir = './models'
    os.makedirs(model_dir, exist_ok=True)
    return model_dir

In [3]:
dataset_dir = get_data_path()
maybe_mount_drive()
os.makedirs(dataset_dir, exist_ok=True)

In [4]:
file_path = os.path.join(dataset_dir, 'train-v2.0.json')

In [5]:
with open(file_path, 'r', encoding='utf-8') as f:
    squad = json.load(f)

In [6]:
records = []
for article in squad['data']:
    for para in article['paragraphs']:
        ctx = para['context']
        for qa in para['qas']:
            answers = [a['text'] for a in qa.get('answers', [])]
            starts  = [a['answer_start'] for a in qa.get('answers', [])]
            ends    = [s + len(t) for s,t in zip(starts, answers)]
            records.append({
                'question': qa['question'],
                'answers': answers,
                'context': ctx,
                'answer_start': starts,
                'answer_end': ends
            })



In [7]:
df = pd.DataFrame(records)
df.head()

,question,answers,context,answer_start,answer_end
0,When did Beyonce start becoming popular?,[in the late 1990s],Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,[269],[286]
1,What areas did Beyonce compete in when she was...,[singing and dancing],Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,[207],[226]
2,When did Beyonce leave Destiny's Child and bec...,[2003],Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,[526],[530]
3,In what city and state did Beyonce grow up?,"[Houston, Texas]",Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,[166],[180]
4,In which decade did Beyonce become famous?,[late 1990s],Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,[276],[286]


In [8]:
print("Total QA pairs:", len(df))

Total QA pairs: 130319


In [9]:
#shuffling
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# only working on subset of 15k row
df_subset = df.head(15000).copy().reset_index(drop=True)

print("Subset size:", df_subset.shape)
df_subset.head()

Subset size: (15000, 5)


,question,answers,context,answer_start,answer_end
0,What year did the global recession that follow...,[2012],It threatened the collapse of large financial ...,[481],[485]
1,what was a popular club in ibiza that started ...,[Amnesia],"But house was also being developed on Ibiza,[c...",[251],[258]
2,In what century did Martin Luther honor Mary a...,[],Although Calvin and Huldrych Zwingli honored M...,[],[]
3,What is the climate like?,[varies from hot and subhumid tropical],"Due to extreme variation in elevation, great v...",[115],[152]
4,How many times has the Queen toured Canada?,[],The Queen addressed the United Nations for a s...,[],[]


## Data Cleaning

Dropping rows where answers are empty

In [10]:
df_subset = df_subset[df_subset['answers'].map(len) > 0].reset_index(drop=True)
print("Rows remaining after drop:", len(df_subset))

Rows remaining after drop: 10020


Removing Extra Whitespaces

In [11]:
def collapse_whitespace(s):
    if isinstance(s, str):
        return re.sub(r'\s+', ' ', s.strip())
    return s

In [12]:
for col in ['question', 'context', 'answers']:
    if col in df_subset.columns:
        df_subset[col] = df_subset[col].apply(collapse_whitespace)

**Lets explore the length of the sequences which will determine some hyperparameters in training the models**

In [13]:
df_subset['question'].str.len().max()

203

In [14]:
df_subset['context'].str.len().max()

3706

We just turn the array of the answers to a string since none have multiple answers

In [15]:
df_subset['answer_text']  = df_subset['answers']  # you already collapsed to the first answer string
df_subset['answer_start'] = df_subset['answer_start'].apply(lambda lst: lst[0])
df_subset['answer_end']   = df_subset['answer_end'].apply(lambda lst: lst[0])

In [16]:
df_subset['answers'].str.len().max()

202

## Embeddings

In [26]:
df_subset['answer_text']  = df_subset['answers']  # you already collapsed to the first answer string
df_subset['answer_start'] = df_subset['answer_start'].apply(lambda lst: lst[0])
df_subset['answer_end']   = df_subset['answer_end'].apply(lambda lst: lst[0])

In [27]:
def whitespace_tokenize_with_spans(text):
    tokens, spans = [], []
    for m in re.finditer(r"\w+|[^\w\s]", text, flags=re.UNICODE):
        tokens.append(m.group())
        spans.append((m.start(), m.end()))
    return tokens, spans

In [28]:
all_texts = df_subset['question'].tolist() + df_subset['context'].tolist()
tokenizer = Tokenizer(oov_token='[UNK]')
tokenizer.fit_on_texts(all_texts)
vocab_size = len(tokenizer.word_index) + 1


In [30]:
max_len = 384


In [31]:
input_ids_list, attn_list = [], []
start_positions, end_positions = [], []

for _, row in df_subset.iterrows():
    q, c = row['question'], row['context']
    ans_s, ans_e = row['answer_start'], row['answer_end']
    
    # tokenize
    q_tokens, _     = whitespace_tokenize_with_spans(q)
    c_tokens, c_spans= whitespace_tokenize_with_spans(c)
    
    # full sequence: Q + [SEP] + C
    seq_tokens = q_tokens + ['[SEP]'] + c_tokens
    
    # to IDs + pad/truncate
    seq_ids = [tokenizer.word_index.get(t, tokenizer.word_index['[UNK]']) 
               for t in seq_tokens]
    seq_ids = seq_ids[:max_len]
    attn   = [1]*len(seq_ids)
    pad_len = max_len - len(seq_ids)
    seq_ids += [0]*pad_len
    attn   += [0]*pad_len
    
    # find which context token covers the answer span
    start_tok = next((i for i,(s,e) in enumerate(c_spans) if s <= ans_s < e), 0)
    end_tok   = next((i for i,(s,e) in enumerate(c_spans) if s < ans_e <= e), start_tok)
    
    sep_off = len(q_tokens) + 1
    start_pos = min(sep_off + start_tok, max_len-1)
    end_pos   = min(sep_off + end_tok,   max_len-1)
    
    input_ids_list.append(seq_ids)
    attn_list.append(attn)
    start_positions.append(start_pos)
    end_positions.append(end_pos)

# 5) Convert to numpy arrays
input_ids_np = np.array(input_ids_list, dtype=np.int32)
attn_np      = np.array(attn_list,      dtype=np.int32)
start_np     = np.array(start_positions, dtype=np.int32)
end_np       = np.array(end_positions,   dtype=np.int32)

# 6) Build a tf.data.Dataset and split 90/10
ds = tf.data.Dataset.from_tensor_slices(
    ({"input_ids": input_ids_np, "attention_mask": attn_np},
     (start_np, end_np))
)
ds = ds.shuffle(len(df_subset), seed=42)
train_size = int(0.9 * len(df_subset))

train_ds = ds.take(train_size).batch(16)
val_ds   = ds.skip(train_size).batch(16)

print(train_ds, val_ds)


<_BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 384), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 384), dtype=tf.int32, name=None)}, (TensorSpec(shape=(None,), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None)))> <_BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 384), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 384), dtype=tf.int32, name=None)}, (TensorSpec(shape=(None,), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None)))>


In [32]:
from tensorflow.keras.layers import Lambda, Reshape

def encoder_block(embed_dim, num_heads, ff_dim):
    x_in = Input(shape=(None, embed_dim))
    mask = Input(shape=(None,), dtype="int32")

    # expand mask to (batch, 1, 1, seq_len) and cast via a Lambda layer
    mask_float = Lambda(
        lambda m: tf.cast(tf.expand_dims(tf.expand_dims(m, axis=1), axis=1), "float32"),
        name="mask_expand"
    )(mask)

    attn = MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_dim
    )(x_in, x_in, attention_mask=mask_float)

    x = LayerNormalization(epsilon=1e-6)(x_in + attn)
    proj = Dense(ff_dim, activation="relu")(x)
    proj = Dense(embed_dim)(proj)
    out  = LayerNormalization(epsilon=1e-6)(x + proj)

    return Model([x_in, mask], out)


In [24]:
# def truncate_context(context: str, ans_start: int, ans_end: int, max_len: int) -> str:
#     """
#     Return a substring of `context` of length up to max_len characters,
#     centered on the character span [ans_start:ans_end], adjusted to word boundaries.
#     """
#     ans_len = ans_end - ans_start
#     extra   = max_len - ans_len
#     pre     = extra // 2
#     post    = extra - pre

#     # ideal window
#     start = ans_start - pre
#     end   = ans_end   + post

#     # shift if off left edge
#     if start < 0:
#         start = 0
#         end   = min(max_len, len(context))

#     # shift if off right edge
#     if end > len(context):
#         end   = len(context)
#         start = max(0, len(context) - max_len)

#     # adjust start backward to nearest whitespace to avoid cutting a word
#     if start > 0 and not context[start].isspace():
#         m = re.search(r'\s', context[:start][::-1])
#         if m:
#             # position of last whitespace before start
#             start = start - m.start()

#     # adjust end forward to nearest whitespace to avoid cutting a word
#     if end < len(context) and not context[end].isspace():
#         m = re.search(r'\s', context[end:])
#         if m:
#             end = end + m.start()

#     # final slice
#     return context[start:end]

# def build_truncated_context(df, max_len: int):
#     """
#     Returns a list of truncated context strings for each row in df,
#     preserving at least the answer span and cutting only at word boundaries.
#     """
#     contexts = []
#     for ctx, starts, ends in zip(df['context'], df['answer_start'], df['answer_end']):
#         # pick the first span
#         s = starts[0]
#         e = ends[0]

#         window = truncate_context(ctx, s, e, max_len)
#         contexts.append(window)

#     return contexts

**Add padding to the encoder and decoder inputs**

## Phase 2 using a transformer

In [33]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, LayerNormalization, MultiHeadAttention
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Lambda, Reshape

embed_dim  = 128
num_heads  = 4
ff_dim     = 512
num_layers = 2

# positional embeddings up to max_len
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, maxlen, dim):
        super().__init__()
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=dim)
    def call(self, x):
        positions = tf.range(tf.shape(x)[1])
        return self.pos_emb(positions)

# one encoder block
def encoder_block(embed_dim, num_heads, ff_dim):
    x_in = Input(shape=(None, embed_dim))
    mask = Input(shape=(None,), dtype="int32")

    # expand mask to (batch, 1, 1, seq_len) and cast via a Lambda layer
    mask_float = Lambda(
        lambda m: tf.cast(tf.expand_dims(tf.expand_dims(m, axis=1), axis=1), "float32"),
        name="mask_expand"
    )(mask)

    attn = MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_dim
    )(x_in, x_in, attention_mask=mask_float)

    x = LayerNormalization(epsilon=1e-6)(x_in + attn)
    proj = Dense(ff_dim, activation="relu")(x)
    proj = Dense(embed_dim)(proj)
    out  = LayerNormalization(epsilon=1e-6)(x + proj)

    return Model([x_in, mask], out)

# inputs
ids = Input(shape=(max_len,), dtype="int32", name="input_ids")
mask= Input(shape=(max_len,), dtype="int32", name="attention_mask")

# embeddings + positional
tok_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim, mask_zero=True)(ids)
pos_emb = PositionalEmbedding(max_len, embed_dim)(ids)
x = tok_emb + pos_emb

# stack encoders
for _ in range(num_layers):
    block = encoder_block(embed_dim, num_heads, ff_dim)
    x = block([x, mask])

# span heads
start_logits = Dense(1)(x)                       # (batch, seq_len, 1)
start_logits = Lambda(lambda t: tf.squeeze(t, -1),
                      name="start_squeeze")(start_logits)

end_logits = Dense(1)(x)
end_logits = Lambda(lambda t: tf.squeeze(t, -1),
                    name="end_squeeze")(end_logits)

start_probs = tf.keras.layers.Activation("softmax", name="start_probs")(start_logits)
end_probs   = tf.keras.layers.Activation("softmax", name="end_probs")(end_logits)

model = Model([ids, mask], [start_probs, end_probs])
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy"
)


In [34]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3
)

Epoch 1/3
564/564 ━━━━━━━━━━━━━━━━━━━━ 25s 25ms/step - end_probs_loss: 4.9682 - loss: 9.9421 - start_probs_loss: 4.9739 - val_end_probs_loss: 4.8659 - val_loss: 9.7311 - val_start_probs_loss: 4.8669
Epoch 2/3
564/564 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - end_probs_loss: 4.9101 - loss: 9.8239 - start_probs_loss: 4.9138 - val_end_probs_loss: 4.9193 - val_loss: 9.8420 - val_start_probs_loss: 4.9220
Epoch 3/3
564/564 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - end_probs_loss: 4.9118 - loss: 9.8246 - start_probs_loss: 4.9128 - val_end_probs_loss: 4.8855 - val_loss: 9.7774 - val_start_probs_loss: 4.8926


In [35]:
# 1) Define the helper again (if you haven’t already in this notebook)
def predict_answer(question, context):
    # tokenize exactly as in preprocess()
    q_tokens, _      = whitespace_tokenize_with_spans(question)
    c_tokens, c_spans= whitespace_tokenize_with_spans(context)
    seq_tokens = q_tokens + ["[SEP]"] + c_tokens

    # convert & pad
    seq_ids = [tokenizer.word_index.get(t, tokenizer.word_index["[UNK]"])
               for t in seq_tokens]
    seq_ids = seq_ids[:max_len] + [0]*(max_len - len(seq_ids))
    attn    = [1]*min(len(seq_tokens),max_len) + [0]*(max_len - min(len(seq_tokens),max_len))

    # run the model
    start_p, end_p = model.predict({
        "input_ids":      tf.constant([seq_ids]),
        "attention_mask": tf.constant([attn])
    })

    # pick the highest‑scoring tokens
    si = tf.argmax(start_p[0]).numpy()
    ei = tf.argmax(end_p[0]).numpy()

    # if the predicted span falls entirely before the sep token, we assume “no answer”
    if ei < len(q_tokens) + 1:
        return ""

    # map back to characters
    c_start = si - (len(q_tokens) + 1)
    c_end   = ei - (len(q_tokens) + 1)
    char_s  = c_spans[c_start][0]
    char_e  = c_spans[c_end][1]
    return context[char_s:char_e]

# 2) Test it on a toy example
question = "Where does John live?"
context  = "John is a software engineer who lives in Cairo and works at GUC."
print("Predicted answer:", predict_answer(question, context))
# → should print: "Cairo"

# 3) Try another one
question = "What is Jane's job?"
context  = "Jane is a biologist at the University of Alexandria. She specializes in marine ecology."
print("Predicted answer:", predict_answer(question, context))
# → should print: "biologist"


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted answer: works
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted answer: Alexandria


In [35]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, max_len: int, embed_dim: int, **kwargs):
        super().__init__(**kwargs)
        # self.supports_masking = True
        self.max_len   = max_len
        self.embed_dim = embed_dim

        pos = np.arange(max_len)[:, np.newaxis]                 
        dim = np.arange(embed_dim)[np.newaxis, :]                
        angle_rates = 1.0 / np.power(10000.0, (2 * (dim//2)) / embed_dim)
        angle_rads  = pos * angle_rates                          
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])        
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])        
        self.pos_encoding = tf.constant(angle_rads[np.newaxis, ...], dtype=tf.float32)
        

    def call(self, x):
        seq_len = tf.shape(x)[1]
        return x + self.pos_encoding[:, :seq_len, :]

    def get_config(self):
        config = super().get_config()
        config.update({
            "max_len": self.max_len,
            "embed_dim": self.embed_dim,
        })
        return config

In [36]:
class Encoder(layers.Layer):
    def __init__(self,embed_dim: int, num_heads: int, ff_dim: int, **kwargs):
        super().__init__(**kwargs)
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embed_dim
        )
        self.layer1 = layers.LayerNormalization()
        self.layer2 = layers.LayerNormalization()
        self.ffn =  tf.keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])

    
    def call(self,pos_matrix, padding_mask, **kwargs):
        att_out= self.attention(query=pos_matrix, value=pos_matrix, key=pos_matrix,attention_mask=padding_mask)
        norm1 = self.layer1(att_out+pos_matrix)
        ff_out = self.ffn(norm1)
        return self.layer2(ff_out)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "ff_dim": self.ff_dim,
        })
        return config

        

In [37]:
class Decoder(layers.Layer):
    def __init__(self, embed_dim: int, num_heads: int, ff_dim: int, dropout=0.1, **kwargs):
        super().__init__(**kwargs)
        # no supports_masking = True
        self.self_mha  = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.cross_mha = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn       = tf.keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.norm1 = layers.LayerNormalization()
        self.norm2 = layers.LayerNormalization()
        self.norm3 = layers.LayerNormalization()

    def call(self, x, enc_out,
             look_ahead_mask=None,
             padding_mask=None,
             training=False):
        # 1) Decoder self‑attention with your 3D look‑ahead+pad mask
        att1 = self.self_mha(
            query=x, value=x, key=x,
            attention_mask=look_ahead_mask,
            training=training
        )
        out1 = self.norm1(x + att1, training=training)

        
        att2 = self.cross_mha(
            query=out1, value=enc_out, key=enc_out,
            attention_mask=padding_mask,
            training=training
        )
        out2 = self.norm2(out1 + att2, training=training)

        
        ffn_out = self.ffn(out2, training=training)
        return self.norm3(out2 + ffn_out, training=training)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "ff_dim": self.ff_dim,
            "embedding_layer": serialize_keras_object(self.token_emb)
            
        })
        return config

    @classmethod
    def from_config(cls, config):
        
        emb_conf = config.pop("embedding_layer")
        
        embedding_layer = deserialize_keras_object(
            emb_conf,
            module_objects=globals(),       # globals() now contains Embedding
            custom_objects=None
        )
        return cls(embedding_layer=embedding_layer, **config)


In [38]:

from tensorflow.keras.utils import serialize_keras_object, deserialize_keras_object

class Seq2SeqTransformer(tf.keras.Model):
    def __init__(self,
                 vocab_size,
                 embed_dim,
                 num_heads,
                 ff_dim,
                 max_enc_in_len,
                 max_a_len,
                 embedding_layer,
                 pad_token_id=0,
                 **kwargs):
        super().__init__(**kwargs)

        # store for serialization
        self.vocab_size     = vocab_size
        self.embed_dim      = embed_dim
        self.num_heads      = num_heads
        self.ff_dim         = ff_dim
        self.max_enc_in_len = max_enc_in_len
        self.max_a_len      = max_a_len
        self.pad_token_id   = pad_token_id

        if embedding_layer is None:
            raise ValueError("`embedding_layer` must be provided")
        self.token_emb   = embedding_layer
        self.pos_emb_enc = PositionalEmbedding(max_enc_in_len, embed_dim)
        self.pos_emb_dec = PositionalEmbedding(max_a_len, embed_dim)
        self.encoder     = Encoder(embed_dim, num_heads, ff_dim)
        self.decoder     = Decoder(embed_dim, num_heads, ff_dim)
        self.final_dense = layers.Dense(vocab_size, activation="softmax")

    def create_padding_mask(self, seq):
        mask = tf.equal(seq, self.pad_token_id)
        return mask[:, None, None, :]

    def create_look_ahead_mask(self, size):
        return 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)

    def call(self, inputs, training=False):
        enc_seq, dec_seq = inputs
        dec_len    = tf.shape(dec_seq)[1]

        enc_padding_mask = self.create_padding_mask(enc_seq)
        dec_pad_bool     = tf.equal(dec_seq, self.pad_token_id)

        look2d      = tf.cast(self.create_look_ahead_mask(dec_len), tf.bool)
        dec_pad_3d  = dec_pad_bool[:, None, :]
        look3d      = look2d[None, :, :]
        self_attn_mask = tf.logical_or(dec_pad_3d, look3d)

        enc_x = self.pos_emb_enc(self.token_emb(enc_seq))
        dec_x = self.pos_emb_dec(self.token_emb(dec_seq))

        enc_out = self.encoder(
            enc_x,
            padding_mask=enc_padding_mask,
            training=training
        )
        dec_out = self.decoder(
            dec_x,
            enc_out,
            look_ahead_mask=self_attn_mask,
            padding_mask=enc_padding_mask,
            training=training
        )
        return self.final_dense(dec_out)

    def get_config(self):
        config = super().get_config()
        config.update({
            "vocab_size":      self.vocab_size,
            "embed_dim":       self.embed_dim,
            "num_heads":       self.num_heads,
            "ff_dim":          self.ff_dim,
            "max_enc_in_len":  self.max_enc_in_len,
            "max_a_len":       self.max_a_len,
            "pad_token_id":    self.pad_token_id,
            "embedding_layer": serialize_keras_object(self.token_emb),
        })
        return config

    @classmethod
    def from_config(cls, config):
        emb_conf = config.pop("embedding_layer")
        embedding_layer = deserialize_keras_object(
            emb_conf,
            module_objects=globals(),
            custom_objects={"Embedding": Embedding}
        )
        return cls(embedding_layer=embedding_layer, **config)


In [39]:
model = Seq2SeqTransformer(
    vocab_size=tokenizer_phase_2.num_words,
    embed_dim=100,
    num_heads=8,
    ff_dim=512,
    max_enc_in_len=MAX_ENCODER_LEN,
    max_a_len=MAX_A_LEN - 1, 
    embedding_layer=embedding_layer,
    pad_token_id=0,
)

### Custom Evaluation metrics

In [40]:
def normalize_answer(s):
    """Lower, strip punctuation/extra whitespace, collapse to tokens."""
    s = s.lower()
    s = re.sub(f"[{re.escape(string.punctuation)}]", " ", s)
    s = " ".join(s.split())
    return s

def exact_match_score(pred, truth):
    return int(normalize_answer(pred) == normalize_answer(truth))

def f1_score(pred, truth):
    pred_tokens = normalize_answer(pred).split()
    truth_tokens = normalize_answer(truth).split()
    if not pred_tokens or not truth_tokens:
        return int(pred_tokens == truth_tokens)
  
    common = {}
    for t in pred_tokens:
        common[t] = common.get(t, 0) + 1
    same = 0
    for t in truth_tokens:
        if common.get(t, 0) > 0:
            same += 1
            common[t] -= 1
    if same == 0:
        return 0.0
    prec = same / len(pred_tokens)
    rec  = same / len(truth_tokens)
    return 2 * prec * rec / (prec + rec)




In [41]:

def decode_batch(seqs, tokenizer, pad_id=0, sos_id=None, eos_id=None):
    texts = tokenizer.sequences_to_texts(seqs)
    clean_texts = []
    for txt in texts:
        tokens = txt.split()
        tokens = [
            t for t in tokens
            if t not in {tokenizer.index_word.get(pad_id, ""), 
                         tokenizer.index_word.get(sos_id, ""), 
                         tokenizer.index_word.get(eos_id, "")}
        ]
        clean_texts.append(" ".join(tokens))
    return clean_texts

class QAEvalCallback(tf.keras.callbacks.Callback):
    def __init__(self, val_ds, tokenizer, pad_id=0, sos_id=None, eos_id=None):
        super().__init__()
        self.val_ds    = val_ds
        self.tokenizer = tokenizer
        self.pad_id    = pad_id
        self.sos_id    = sos_id
        self.eos_id    = eos_id

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        total_em = 0.0
        total_f1 = 0.0
        count    = 0

        for (enc_batch, dec_in_batch), dec_tar_batch, _ in self.val_ds:
            logits = self.model((enc_batch, dec_in_batch), training=False)
            pred_ids = tf.argmax(logits, axis=-1).numpy()

            preds = decode_batch(pred_ids,
                                 tokenizer=self.tokenizer,
                                 pad_id=self.pad_id,
                                 sos_id=self.sos_id,
                                 eos_id=self.eos_id)
            reals = decode_batch(dec_tar_batch.numpy(),
                                 tokenizer=self.tokenizer,
                                 pad_id=self.pad_id,
                                 sos_id=self.sos_id,
                                 eos_id=self.eos_id)

            for p_str, r_str in zip(preds, reals):
                total_em += exact_match_score(p_str, r_str)
                total_f1 += f1_score(p_str, r_str)
                count   += 1

        val_em = 100.0 * total_em / count
        val_f1 = 100.0 * total_f1 / count
        logs["val_em"] = val_em
        logs["val_f1"] = val_f1
        print(f" — val_EM: {val_em:.2f}%  — val_F1: {val_f1:.2f}%")


### Model training

In [42]:
dummy_enc = tf.zeros((1, MAX_ENCODER_LEN), dtype=tf.int32)
dummy_dec = tf.zeros((1,MAX_A_LEN - 1 ), dtype=tf.int32)
_ = model((dummy_enc, dummy_dec), training=False)
model.summary()


model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=[]    
)


Model: "seq2_seq_transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ pretrained_embedding (Embedding)     │ ?                           │       5,729,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ positional_embedding                 │ ?                           │               0 │
│ (PositionalEmbedding)                │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ positional_embedding_1               │ ?                           │               0 │
│ (PositionalEmbedding)                │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ encoder (Encoder)                    │ ?                           │         425,912 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ decoder (Decoder)                    │ ?                           │         748,612 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (1, 31, 50000)              │       5,050,000 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,953,724 (45.60 MB)

 Trainable params: 6,224,524 (23.74 MB)

 Non-trainable params: 5,729,200 (21.86 MB)

In [48]:
MODEL_DIR       = get_model_dir()
CHECKPOINT_PATH = os.path.join(MODEL_DIR, 'best_transformer_1.keras')

checkpoint_cb = ModelCheckpoint(
    filepath=CHECKPOINT_PATH,
    monitor='val_f1',      
    save_best_only=True,
    mode='max',
    verbose=1
)


qa_eval_cb = QAEvalCallback(
    val_ds,
    tokenizer_phase_2,
    pad_id=pad_id,
    sos_id=sos_id,
    eos_id=eos_id
)
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[qa_eval_cb,checkpoint_cb]
)

print(f"Best model will be saved to: {CHECKPOINT_PATH}")


Epoch 1/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.0716 — val_EM: 0.00%  — val_F1: 11.97%

Epoch 1: val_f1 improved from -inf to 11.97097, saving model to /kaggle/working/models/best_transformer_1.keras
141/141 ━━━━━━━━━━━━━━━━━━━━ 13s 92ms/step - loss: 0.0716 - val_loss: 0.4020 - val_em: 0.0000e+00 - val_f1: 11.9710
Epoch 2/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.0432 — val_EM: 0.00%  — val_F1: 12.38%

Epoch 2: val_f1 improved from 11.97097 to 12.38409, saving model to /kaggle/working/models/best_transformer_1.keras
141/141 ━━━━━━━━━━━━━━━━━━━━ 13s 93ms/step - loss: 0.0432 - val_loss: 0.4000 - val_em: 0.0000e+00 - val_f1: 12.3841
Epoch 3/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.0259 — val_EM: 0.00%  — val_F1: 12.41%

Epoch 3: val_f1 improved from 12.38409 to 12.41223, saving model to /kaggle/working/models/best_transformer_1.keras
141/141 ━━━━━━━━━━━━━━━━━━━━ 13s 93ms/step - loss: 0.0259 - val_loss: 0.4070 - val_em: 0.0000e+00 - val_f1: 12.4122

In [50]:
best_model = load_model(
    '/kaggle/working/models/best_transformer_1.keras',
    custom_objects={
        "Seq2SeqTransformer": Seq2SeqTransformer,
        "PositionalEmbedding": PositionalEmbedding,
        "Encoder": Encoder,
        "Decoder": Decoder,
        "Embedding": Embedding,   
    }
)

best_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=[]
)


history = best_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=[qa_eval_cb, checkpoint_cb]
)

Epoch 1/20


W0000 00:00:1745344500.606035     100 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


140/141 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.0128

W0000 00:00:1745344514.557092     101 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - loss: 0.0129

W0000 00:00:1745344519.146207      99 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
W0000 00:00:1745344520.341257      98 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


 — val_EM: 0.00%  — val_F1: 12.54%

Epoch 1: val_f1 did not improve from 12.66712
141/141 ━━━━━━━━━━━━━━━━━━━━ 33s 135ms/step - loss: 0.0129 - val_loss: 0.3610 - val_em: 0.0000e+00 - val_f1: 12.5432
Epoch 2/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.0095 — val_EM: 0.00%  — val_F1: 12.51%

Epoch 2: val_f1 did not improve from 12.66712
141/141 ━━━━━━━━━━━━━━━━━━━━ 13s 89ms/step - loss: 0.0095 - val_loss: 0.3694 - val_em: 0.0000e+00 - val_f1: 12.5099
Epoch 3/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.0087 — val_EM: 0.00%  — val_F1: 12.52%

Epoch 3: val_f1 did not improve from 12.66712
141/141 ━━━━━━━━━━━━━━━━━━━━ 13s 89ms/step - loss: 0.0087 - val_loss: 0.3768 - val_em: 0.0000e+00 - val_f1: 12.5230
Epoch 4/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.0081 — val_EM: 0.00%  — val_F1: 12.49%

Epoch 4: val_f1 did not improve from 12.66712
141/141 ━━━━━━━━━━━━━━━━━━━━ 13s 89ms/step - loss: 0.0081 - val_loss: 0.3859 - val_em: 0.0000e+00 - val_f1: 12.4909
Epoch 

In [77]:


print("  • # train batches:", tf.data.experimental.cardinality(train_ds).numpy())
print("  • # val   batches:", tf.data.experimental.cardinality(val_ds).numpy())


for (enc_batch, decin_batch), dec_tar_batch, weight_batch in train_ds.take(1):
    print("Encoder shape:", enc_batch.shape)       # (B, enc_len)
    print("Decoder‑in shape:", decin_batch.shape)  # (B, ans_len)
    print("Decoder‑tar shape:", dec_tar_batch.shape)
    print("Weights shape:", weight_batch.shape)
    
    enc_ids = enc_batch[0].numpy().tolist()
    decin_ids = decin_batch[0].numpy().tolist()
    detar_ids = dec_tar_batch[0].numpy().tolist()
    
    
    enc_ids = [i for i in enc_ids if i != pad_id]
    decin_ids = [i for i in decin_ids if i not in (pad_id, sos_id, eos_id)]
    detar_ids = [i for i in detar_ids if i not in (pad_id, sos_id, eos_id)]
    
    q_c_text = tokenizer_phase_2.sequences_to_texts([enc_ids])[0]
    inp_ans  = tokenizer_phase_2.sequences_to_texts([decin_ids])[0]
    tar_ans  = tokenizer_phase_2.sequences_to_texts([detar_ids])[0]
    
    print("\nSample #1")
    print("  Q+SEP+C →", q_c_text[:200] + "…")
    print("  Decoder‑in →", inp_ans)
    print("  Decoder‑tar→", tar_ans)
    break


  • # train batches: 141
  • # val   batches: 16
Encoder shape: (64, 408)
Decoder‑in shape: (64, 31)
Decoder‑tar shape: (64, 31)
Weights shape: (64, 31)

Sample #1
  Q+SEP+C → what are two of wright's designs [SEP] architects such as frank lloyd wright developed organic architecture in which the form was defined by its environment and purpose with an aim to promote harmony …
  Decoder‑in → robie house and fallingwater
  Decoder‑tar→ robie house and fallingwater


In [66]:


def show_errors_greedy(model, dataset, tokenizer, max_encoder_len, max_answer_len, sep_id, pad_id, sos_id, eos_id, n=5):
    it = dataset.as_numpy_iterator()
    samples = random.sample(list(it), n)
    for (enc, _), dec_tar, _ in samples:
        enc_row = [i for i in enc[0].tolist() if i != pad_id]
        sep_index = enc_row.index(sep_id)
        q_ids = enc_row[:sep_index]
        c_ids = enc_row[sep_index+1:]
        q_txt = tokenizer.sequences_to_texts([q_ids])[0]
        c_txt = tokenizer.sequences_to_texts([c_ids])[0]
        dec_input = [sos_id]
        for _ in range(max_answer_len):
            dec_pad = pad_sequences([dec_input], maxlen=max_answer_len, padding="post", value=pad_id)
            logits = model((enc, dec_pad), training=False)
            next_id = int(tf.argmax(logits[0, len(dec_input)-1]).numpy())
            if next_id == eos_id:
                break
            dec_input.append(next_id)
        pred_ids = [i for i in dec_input[1:] if i not in (pad_id, sos_id, eos_id)]
        true_ids = [i for i in dec_tar[0].tolist() if i not in (pad_id, sos_id, eos_id)]
        pred_txt = tokenizer.sequences_to_texts([pred_ids])[0]
        true_txt = tokenizer.sequences_to_texts([true_ids])[0]
        print(f"Q:  {q_txt!r}")
        print(f"C:  {c_txt[:200]!r}…")
        print(f"GT → {true_txt!r}")
        print(f"PR → {pred_txt!r}")
        print("-"*80)

show_errors_greedy(best_model, val_ds, tokenizer_phase_2, MAX_ENCODER_LEN, MAX_A_LEN-1, sep_id, pad_id, sos_id, eos_id, n=5)


Q:  'what was the less expensive version of the macintosh se that was offered until 2001'
C:  'in response apple introduced a range of relatively inexpensive macs in october 1990 the macintosh classic essentially a less expensive version of the macintosh se was the least expensive mac offered u'…
GT → 'the macintosh classic'
PR → '60–80'
--------------------------------------------------------------------------------
Q:  'what does federalism mean in europe'
C:  'in contrast europe has a greater history of unitary states than north america thus european federalism argues for a weaker central government relative to a unitary state the modern american usage of t'…
GT → 'weaker central government relative to a unitary state'
PR → '60–80'
--------------------------------------------------------------------------------
Q:  'which group of people were now considered to be politically equal with the plebeians'
C:  'the plebeians had finally achieved political equality with the patricians howe

In [67]:
import pandas as pd
from collections import Counter

# Suppose decoder_targets is your unbatched numpy array of shape (N, seq_len)
# You padded to MAX_A_LEN, so decoder_targets exists as detar before train_test_split.
all_answers = []
for seq in decoder_targets[:500]:
    clean_ids = [i for i in seq if i not in (pad_id, sos_id, eos_id)]
    text = tokenizer_phase_2.sequences_to_texts([clean_ids])[0]
    all_answers.append(text)

answer_counts = Counter(all_answers)
df_top = pd.DataFrame(answer_counts.items(), columns=['answer', 'count']) \
    .sort_values('count', ascending=False) \
    .reset_index(drop=True)

print(df_top.head(20))


                                               answer  count
0                                                  45      2
1                                                1988      2
2                                               1 000      2
3                                               india      2
4                                           the bible      2
5                                                2010      2
6                                               eight      2
7                                            couplets      1
8                digitize and offer nara video online      1
9                          banking financial services      1
10            pollen either fails to reach the stigma      1
11            the communist party of the soviet union      1
12                                               rich      1
13                                      louis agassiz      1
14                                                 56      1
15                      

## Post Processing

In [75]:
def greedy_decode(model,
                  question: str,
                  context: str,
                  tokenizer,
                  max_encoder_len: int,
                  max_answer_len: int,
                  sep_token_id: int,
                  pad_token_id: int,
                  sos_token_id: int,
                  eos_token_id: int):
    q_ids = tokenizer.texts_to_sequences([question])[0]
    c_ids = tokenizer.texts_to_sequences([context])[0]
    enc_seq = q_ids + [sep_token_id] + c_ids
    enc_input = pad_sequences(
        [enc_seq],
        maxlen=max_encoder_len,
        padding='post',
        truncating='post',
        value=pad_token_id
    )

   
    dec_input = [sos_token_id]

    for _ in range(max_answer_len):
        dec_pad = pad_sequences(
            [dec_input],
            maxlen=max_answer_len,
            padding='post',
            truncating='post',
            value=pad_token_id
        )

        preds = model((enc_input, dec_pad), training=False)
        next_id = int(tf.argmax(preds[0, len(dec_input)-1]).numpy())
        if next_id == eos_token_id:
            break

        dec_input.append(next_id)

    decoded_tokens = [tokenizer.index_word.get(i, "") for i in dec_input[1:]]
    return " ".join(decoded_tokens)


sos_id = tokenizer_phase_2.word_index["[SOS]"]
eos_id = tokenizer_phase_2.word_index["[EOS]"]
sep_id = tokenizer_phase_2.word_index["[SEP]"]
pad_id = 0

question = "What is the capital of France?"
context  = ("France is a country in Western Europe. "
            "Its capital city is Paris, known for the Eiffel Tower.")

answer = greedy_decode(
    best_model,
    question,
    context,
    tokenizer_phase_2,
    max_encoder_len=MAX_ENCODER_LEN,
    max_answer_len=MAX_A_LEN-1,
    sep_token_id=sep_id,
    pad_token_id=pad_id,
    sos_token_id=sos_id,
    eos_token_id=eos_id
)
print("Answer:", answer)


Answer: 60–80


In [76]:
question = "What is my name?"
context = "My name is ahmed and please my name is ahmed"

answer2 = greedy_decode(
    best_model,
    question,
    context,
    tokenizer_phase_2,
    max_encoder_len=MAX_ENCODER_LEN,
    max_answer_len=MAX_A_LEN-1,
    sep_token_id=sep_id,
    pad_token_id=pad_id,
    sos_token_id=sos_id,
    eos_token_id=eos_id
)
print("Answer: ", answer2 )

Answer:  60–80


## Inference using the best model

In [83]:
from datasets import load_dataset
squad_dev = load_dataset("squad_v2", split="validation")

README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [86]:
import pandas as pd

# 1) Pull out the raw columns
contexts = ans_dev["context"]
answer_dicts = ans_dev["answers"]  # this is a list of {"text": [...], "answer_start": [...]}

# 2) Split into two parallel lists: starts and texts
answer_starts = [d["answer_start"] for d in answer_dicts]
answer_texts  = [d["text"]         for d in answer_dicts]

# 3) Compute end positions for each text span
answer_ends = [
    [s + len(t) for s, t in zip(starts, texts)]
    for starts, texts in zip(answer_starts, answer_texts)
]

# 4) Build your DataFrame
df_dev = pd.DataFrame({
    "context":      contexts,
    "answer_start": answer_starts,
    "answer_end":   answer_ends,
})

# Now your previous code will work:
contexts_trunc = build_truncated_context(df_dev, max_len=2000)
questions     = ans_dev["question"]
answers       = [texts[0] for texts in answer_texts]  # take first text per example


Filter:   0%|          | 0/11873 [00:00<?, ? examples/s]

TypeError: list indices must be integers or slices, not str